In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import spearmanr, kendalltau, pearsonr

# Calculate the 5-day percent change for a given ticker
def five_day_percent_change(dates, ticker):
    stock = yf.Ticker(ticker)
    percent_changes = []

    for date_str in dates:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        start_date = date.strftime('%Y-%m-%d')
        end_date = (date + timedelta(days=5)).strftime('%Y-%m-%d')

        history = stock.history(start=start_date, end=end_date)
        if len(history) >= 2:
            start_price = history.iloc[0]['Close']
            end_price = history.iloc[-1]['Close']
            percent_change = (end_price - start_price) / start_price * 100
            percent_changes.append(percent_change)
        else:
            percent_changes.append(None)

    return percent_changes

# Calculate the 5-day percent change for multiple tickers
def five_day_percent_changes_multiple_tickers(dates, tickers_input):
    tickers = tickers_input.split(", ")
    
    all_percent_changes = {}
    for ticker in tickers:
        percent_changes = five_day_percent_change(dates, ticker)
        all_percent_changes[ticker] = percent_changes
    return all_percent_changes

# Calculate the average of the spearman, kendall and pearson correlations
def average_correlations(list1, list2):
    spearman_corr, _ = spearmanr(list1, list2)
    kendall_corr, _ = kendalltau(list1, list2)
    pearson_corr, _ = pearsonr(list1, list2)

    average_corr = np.mean([spearman_corr, kendall_corr, pearson_corr])
    return average_corr

# Calculate the average correlations for multiple tickers
def average_correlations_multiple_companies(list1, companies_lists):
    company_correlations = {}

    for company, list2 in companies_lists.items():
        average_corr = average_correlations(list1, list2)
        company_correlations[company] = average_corr

    overall_average_corr = np.mean(list(company_correlations.values()))
    return company_correlations, overall_average_corr

# Get user inputs
dates = [date.strip("'") for date in input("Enter the list of dates (comma-separated, with single quotes): ").split(', ')]
eps_values = list(map(float, input("Enter the list of EPS values (comma-separated): ").split(', ')))
tickers_input = input("Enter 5 tickers separated by commas: ")

# Calculate percent changes for the provided tickers
all_percent_changes = five_day_percent_changes_multiple_tickers(dates, tickers_input)

# Calculate average correlations
company_correlations, overall_average_corr = average_correlations_multiple_companies(eps_values, all_percent_changes)

# Print results
for company, average_corr in company_correlations.items():
    print(f"Average correlation for {company}: {average_corr:.4f}")

print(f"Overall average correlation: {overall_average_corr:.4f}")
